In [1]:
import numpy
import pandas as pd
import os
import csv
import ast
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

from wquantiles import quantile
from openfisca_survey_manager.utils import asof

from openfisca_france_indirect_taxation import FranceIndirectTaxationTaxBenefitSystem
from openfisca_france_indirect_taxation.examples.utils_example import (
    wavg,
    collapse,
    dataframe_by_group,
    graph_builder_bar,
    df_weighted_average_grouped)
from openfisca_france_indirect_taxation.build_survey_data.utils import weighted_sum
from openfisca_france_indirect_taxation.almost_ideal_demand_system.utils import add_niveau_vie_decile
from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.calibration import get_inflators_by_year_energy
from openfisca_france_indirect_taxation.utils import assets_directory, get_input_data_frame

In [2]:
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn import (
    new_get_bdf_aggregates,
    new_get_cn_aggregates,
    new_get_inflators_bdf_to_cn,
    new_get_inflators_cn_to_cn,
    new_get_inflators,
    new_get_inflators_by_year)

In [3]:
data_year = 2017
inflators_by_year = new_get_inflators_by_year(rebuild = True, year_range = range(2017, 2025), data_year = data_year)
simulated_variables = ['tva_taux_plein',
'tva_taux_intermediaire',
 'tva_taux_reduit',
 'tva_taux_super_reduit',
 'tva_total',
 'depenses_tva_exonere',
 'depenses_tva_taux_plein',
 'depenses_tva_taux_intermediaire',
 'depenses_tva_taux_reduit',
 'depenses_tva_taux_super_reduit',
 'rev_disponible',
 'rev_disp_yc_loyerimpute',
 'loyer_impute',
 'depenses_tot',
 'depenses_totales',
 'pondmen',
]

agregates = pd.DataFrame(columns = simulated_variables)
for year in range(2017,2025) :
    inflation_kwargs = dict(inflator_by_variable = inflators_by_year[year])
    survey_scenario = SurveyScenario.create(
        inflation_kwargs = inflation_kwargs,
        year = year,
        data_year = data_year
        )
    df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
    agregates.loc[year] = df_sum.sum()

c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
c:\users\veve1\onedrive\documents\ensae 3a\memoire mie\openfisca-france-indirect-taxation\openfisca_france_indirect_taxation\projects\TVA_Herve_IPP\new_calage_bdf_cn.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_bdf_postes_cn[poste] = 0


In [4]:
agregates

,tva_taux_plein,tva_taux_intermediaire,tva_taux_reduit,tva_taux_super_reduit,tva_total,depenses_tva_exonere,depenses_tva_taux_plein,depenses_tva_taux_intermediaire,depenses_tva_taux_reduit,depenses_tva_taux_super_reduit,rev_disponible,rev_disp_yc_loyerimpute,loyer_impute,depenses_tot,depenses_totales,pondmen
2017,7.845328e+10,1.441827e+10,7.862741e+09,2.916588e+08,1.010260e+11,2.136593e+11,4.707197e+11,1.586010e+11,1.508217e+11,1.418017e+10,1.197964e+12,1.387077e+12,1.891127e+11,1.007982e+12,7.943225e+11,8.445859e+10
2018,8.068022e+10,1.514731e+10,7.973502e+09,2.906346e+08,1.040917e+11,2.231091e+11,4.840814e+11,1.666204e+11,1.529463e+11,1.413038e+10,1.234271e+12,1.426805e+12,1.925339e+11,1.040887e+12,8.177784e+11,8.445859e+10
2019,8.205242e+10,1.592258e+10,8.136745e+09,2.915240e+08,1.064033e+11,2.265549e+11,4.923145e+11,1.751484e+11,1.560776e+11,1.417362e+10,1.278664e+12,1.475381e+12,1.967170e+11,1.064269e+12,8.377141e+11,8.445859e+10
2020,7.561451e+10,1.126653e+10,8.580251e+09,2.828792e+08,9.574417e+10,2.245522e+11,4.536871e+11,1.239319e+11,1.645848e+11,1.375332e+10,1.289747e+12,1.489995e+12,2.002478e+11,9.805093e+11,7.559571e+11,8.445859e+10
2021,8.325562e+10,1.295723e+10,8.638843e+09,2.987876e+08,1.051505e+11,2.342183e+11,4.995337e+11,1.425295e+11,1.657087e+11,1.452677e+10,1.358404e+12,1.562187e+12,2.037834e+11,1.056517e+12,8.222988e+11,8.445859e+10
2022,8.861955e+10,1.796930e+10,9.029455e+09,3.059662e+08,1.159243e+11,2.479648e+11,5.317173e+11,1.976623e+11,1.732013e+11,1.487579e+10,1.434894e+12,1.642992e+12,2.080976e+11,1.165422e+12,9.174568e+11,8.445859e+10
2023,9.282289e+10,1.996829e+10,9.769284e+09,3.112216e+08,1.228717e+11,2.906248e+11,5.569374e+11,2.196512e+11,1.873926e+11,1.513130e+10,1.560302e+12,1.773999e+12,2.136968e+11,1.269737e+12,9.791125e+11,8.445859e+10
2024,9.549461e+10,2.054304e+10,1.005047e+10,3.201795e+08,1.264083e+11,2.989898e+11,5.729677e+11,2.259734e+11,1.927863e+11,1.556682e+10,1.605212e+12,1.825060e+12,2.198476e+11,1.306284e+12,1.007294e+12,8.445859e+10


In [4]:
cn_agregates_by_year = pd.DataFrame()
for year in range(2017,2024):
    cn_agregates = new_get_cn_aggregates(year)
    cn_agregates.drop(['poste_02_4', 'poste_04_4_3','rev_disp_yc_loyerimpute','rev_disponible'], axis= 0, inplace = True)
    cn_agregates_by_year = pd.concat([cn_agregates_by_year,pd.DataFrame(cn_agregates.sum())])
cn_agregates_by_year

,0
conso_CN_2017,1.197358e+12
conso_CN_2018,1.233693e+12
conso_CN_2019,1.261263e+12
conso_CN_2020,1.181017e+12
conso_CN_2021,1.260578e+12
conso_CN_2022,1.373821e+12
conso_CN_2023,1.483760e+12


In [5]:
agregates['depenses_tot'] + agregates['loyer_impute']

2017    1.197095e+12
2018    1.233421e+12
2019    1.260986e+12
2020    1.180757e+12
2021    1.260300e+12
2022    1.373519e+12
2023    1.483434e+12
2024    1.526132e+12
dtype: float64

**Par postes agrégés**

In [41]:
from openfisca_france_indirect_taxation.projects.TVA_Herve_IPP.new_calage_bdf_cn_by_postes_agreges import (get_bdf_aggregates,
    get_cn_aggregates,
    get_inflators_bdf_to_cn,
    get_inflators_cn_to_cn,
    get_inflators_by_year)

In [42]:
aggregates_13_postes = get_cn_aggregates(2021)
aggregates_13_postes


c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
c:\Users\veve1\miniconda3\envs\openfiscaenv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,conso_CN_2021
Code,
poste_01,1.623663e+11
poste_02,5.157860e+10
poste_03,4.251240e+10
poste_04,3.756261e+11
poste_05,5.797690e+10
poste_06,5.179090e+10
poste_07,1.501592e+11
poste_08,5.183640e+10
poste_09,8.225710e+10


In [47]:
data_year = 2017
inflators = new_get_inflators(2021,2017)
simulated_variables = ["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_10", "poste_agrege_11", "poste_agrege_12", "poste_agrege_13"]

postes_agreges = pd.DataFrame(columns=["poste_agrege_0{}".format(i) for i in range(1, 10)] + ["poste_agrege_10", "poste_agrege_11", "poste_agrege_12", "poste_agrege_13"])
inflation_kwargs = dict(inflator_by_variable = inflators)
survey_scenario = SurveyScenario.create(
    inflation_kwargs = inflation_kwargs,
    year = 2021,
    data_year = data_year
    )
df_sum = dataframe_by_group(survey_scenario, category = 'niveau_vie_decile', variables = simulated_variables, aggfunc = 'sum')
postes_agreges = df_sum.sum()

c:\users\veve1\onedrive\documents\ensae 3a\memoire mie\openfisca-france-indirect-taxation\openfisca_france_indirect_taxation\projects\TVA_Herve_IPP\new_calage_bdf_cn.py:151: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data_bdf_postes_cn[poste] = 0


In [48]:
postes_agreges.T

poste_agrege_01    1.623306e+11
poste_agrege_02    4.709131e+10
poste_agrege_03    4.250286e+10
poste_agrege_04    1.655719e+11
poste_agrege_05    5.796394e+10
poste_agrege_06    5.177962e+10
poste_agrege_07    1.501256e+11
poste_agrege_08    5.182510e+10
poste_agrege_09    8.223902e+10
poste_agrege_10    9.515843e+09
poste_agrege_11    7.877509e+10
poste_agrege_12    7.786302e+10
poste_agrege_13    7.893317e+10
dtype: float64

In [49]:
aggregates_13_postes

,conso_CN_2021
Code,
poste_01,1.623663e+11
poste_02,5.157860e+10
poste_03,4.251240e+10
poste_04,3.756261e+11
poste_05,5.797690e+10
poste_06,5.179090e+10
poste_07,1.501592e+11
poste_08,5.183640e+10
poste_09,8.225710e+10


In [ ]:
agregates

,tva_taux_plein,tva_taux_intermediaire,tva_taux_reduit,tva_taux_super_reduit,tva_total,depenses_tva_exonere,depenses_tva_taux_plein,depenses_tva_taux_intermediaire,depenses_tva_taux_reduit,depenses_tva_taux_super_reduit,rev_disponible,rev_disp_yc_loyerimpute,loyer_impute,depenses_tot,depenses_totales,pondmen
2017,7.845328e+10,1.441827e+10,7.862741e+09,2.916588e+08,1.010260e+11,2.136593e+11,4.707197e+11,1.586010e+11,1.508217e+11,1.418017e+10,1.197964e+12,1.387077e+12,1.891127e+11,1.007982e+12,7.943225e+11,8.445859e+10
2018,8.068022e+10,1.514731e+10,7.973502e+09,2.906346e+08,1.040917e+11,2.231091e+11,4.840814e+11,1.666204e+11,1.529463e+11,1.413038e+10,1.234271e+12,1.426805e+12,1.925339e+11,1.040887e+12,8.177784e+11,8.445859e+10
2019,8.205242e+10,1.592258e+10,8.136745e+09,2.915240e+08,1.064033e+11,2.265549e+11,4.923145e+11,1.751484e+11,1.560776e+11,1.417362e+10,1.278664e+12,1.475381e+12,1.967170e+11,1.064269e+12,8.377141e+11,8.445859e+10
2020,7.561451e+10,1.126653e+10,8.580251e+09,2.828792e+08,9.574417e+10,2.245522e+11,4.536871e+11,1.239319e+11,1.645848e+11,1.375332e+10,1.289747e+12,1.489995e+12,2.002478e+11,9.805093e+11,7.559571e+11,8.445859e+10
2021,8.325562e+10,1.295723e+10,8.638843e+09,2.987876e+08,1.051505e+11,2.342183e+11,4.995337e+11,1.425295e+11,1.657087e+11,1.452677e+10,1.358404e+12,1.562187e+12,2.037834e+11,1.056517e+12,8.222988e+11,8.445859e+10
2022,8.861955e+10,1.796930e+10,9.029455e+09,3.059662e+08,1.159243e+11,2.479648e+11,5.317173e+11,1.976623e+11,1.732013e+11,1.487579e+10,1.434894e+12,1.642992e+12,2.080976e+11,1.165422e+12,9.174568e+11,8.445859e+10
2023,9.282289e+10,1.996829e+10,9.769284e+09,3.112216e+08,1.228717e+11,2.906248e+11,5.569374e+11,2.196512e+11,1.873926e+11,1.513130e+10,1.560302e+12,1.773999e+12,2.136968e+11,1.269737e+12,9.791125e+11,8.445859e+10
2024,9.549461e+10,2.054304e+10,1.005047e+10,3.201795e+08,1.264083e+11,2.989898e+11,5.729677e+11,2.259734e+11,1.927863e+11,1.556682e+10,1.605212e+12,1.825060e+12,2.198476e+11,1.306284e+12,1.007294e+12,8.445859e+10
